## 실습1. 문장 간의 유사성

#### 샘플 예제

In [1]:
doc_list = ['if you take the blue pill, the story ends' ,
            'if you take the red pill, you stay in Wonderland',
            'if you take the red pill, I show you how deep the rabbit hole goes']

#### TF-IDF 피처 벡터화

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tfidf_vect = TfidfVectorizer()
# tfidf_vect.fit(doc_list)
# ftr_mat = tfidf_vect.transform(doc_list)
ftr_mat = tfidf_vect.fit_transform(doc_list)
print(f'크기 : {ftr_mat.shape}')
print(f'단어집합 : \n{tfidf_vect.vocabulary_}')
print(f'피처벡터 : \n{ftr_mat.toarray()}')

크기 : (3, 18)
단어집합 : 
{'if': 6, 'you': 17, 'take': 14, 'the': 15, 'blue': 0, 'pill': 8, 'story': 13, 'ends': 2, 'red': 10, 'stay': 12, 'in': 7, 'wonderland': 16, 'show': 11, 'how': 5, 'deep': 1, 'rabbit': 9, 'hole': 4, 'goes': 3}
피처벡터 : 
[[0.4155636  0.         0.4155636  0.         0.         0.
  0.24543856 0.         0.24543856 0.         0.         0.
  0.         0.4155636  0.24543856 0.49087711 0.         0.24543856]
 [0.         0.         0.         0.         0.         0.
  0.23402865 0.39624495 0.23402865 0.         0.3013545  0.
  0.39624495 0.         0.23402865 0.23402865 0.39624495 0.4680573 ]
 [0.         0.30985601 0.         0.30985601 0.30985601 0.30985601
  0.18300595 0.         0.18300595 0.30985601 0.23565348 0.30985601
  0.         0.         0.18300595 0.3660119  0.         0.3660119 ]]


#### 피처 벡터 희소행렬을 dense matrix로 변환

TF-IDF로 피처 벡터화한 행렬은 희소행렬이므로 dense 행렬로 변환한 후 각각을 배열로 변환
- 피처벡터객체.todense()

In [7]:
ftr_mat_dense = ftr_mat.todense()
ftr_mat_dense

matrix([[0.4155636 , 0.        , 0.4155636 , 0.        , 0.        ,
         0.        , 0.24543856, 0.        , 0.24543856, 0.        ,
         0.        , 0.        , 0.        , 0.4155636 , 0.24543856,
         0.49087711, 0.        , 0.24543856],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.23402865, 0.39624495, 0.23402865, 0.        ,
         0.3013545 , 0.        , 0.39624495, 0.        , 0.23402865,
         0.23402865, 0.39624495, 0.4680573 ],
        [0.        , 0.30985601, 0.        , 0.30985601, 0.30985601,
         0.30985601, 0.18300595, 0.        , 0.18300595, 0.30985601,
         0.23565348, 0.30985601, 0.        , 0.        , 0.18300595,
         0.3660119 , 0.        , 0.3660119 ]])

### 1. 코사인 유사도

- 방법1. cos_similarity() 함수 사용 : 두 문장의 코사인 유사도 계산


- 방법2. 사이킷런의 코사인 유사도 측정 API 사용
    -  sklearn.metrics.pairwise.cosine_similarity(비교기준 문서의 피처행렬, 비교되는 문서의 피처행렬)
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

#### 1) 코사인 유사도 함수 cos_similarity() 사용

In [10]:
import numpy as np

def cos_similarity(v1, v2):
    dot_prod = np.dot(v1, v2)
    L2norm_v1 = np.sqrt(np.sum(np.square(v1)))
    L2norm_v2 = np.sqrt(np.sum(np.square(v2)))
    L2norm = L2norm_v1 * L2norm_v2
    return dot_prod / L2norm

In [12]:
vect1 = np.array(ftr_mat_dense[0]).reshape(-1,)
vect2 = np.array(ftr_mat_dense[1]).reshape(-1,)

print(f'문장1 : {doc_list[0]}')
print(f'문장2 : {doc_list[1]}')

print(f'문장1과 문장2의 코사인 유사도 = {cos_similarity(vect1, vect2):.3f}')

문장1 : if you take the blue pill, the story ends
문장2 : if you take the red pill, you stay in Wonderland
문장1과 문장2의 코사인 유사도 = 0.402


In [13]:
vect1 = np.array(ftr_mat_dense[0]).reshape(-1,)
vect3 = np.array(ftr_mat_dense[2]).reshape(-1,)

print(f'문장1 : {doc_list[0]}')
print(f'문장3 : {doc_list[2]}')

print(f'문장1과 문장3의 코사인 유사도 = {cos_similarity(vect1, vect3):.3f}')

문장1 : if you take the blue pill, the story ends
문장3 : if you take the red pill, I show you how deep the rabbit hole goes
문장1과 문장3의 코사인 유사도 = 0.404


In [14]:
vect2 = np.array(ftr_mat_dense[1]).reshape(-1,)
vect3 = np.array(ftr_mat_dense[2]).reshape(-1,)

print(f'문장2 : {doc_list[1]}')
print(f'문장3 : {doc_list[2]}')

print(f'문장2와 문장3의 코사인 유사도 = {cos_similarity(vect2, vect3):.3f}')

문장2 : if you take the red pill, you stay in Wonderland
문장3 : if you take the red pill, I show you how deep the rabbit hole goes
문장2와 문장3의 코사인 유사도 = 0.456


In [15]:
def doc_cos_sim(doc, i=0, j=1):
    vect1 = np.array(ftr_mat_dense[i]).reshape(-1,)
    vect2 = np.array(ftr_mat_dense[j]).reshape(-1,)

    print(f'문장{i+1} : {doc_list[i]}')
    print(f'문장{j+1} : {doc_list[j]}')

    print(f'문장{i+1}과 문장{j+1}의 코사인 유사도 = {cos_similarity(vect1, vect2):.3f}')

In [16]:
doc_cos_sim(doc_list, 0, 1)

문장1 : if you take the blue pill, the story ends
문장2 : if you take the red pill, you stay in Wonderland
문장1과 문장2의 코사인 유사도 = 0.402


#### 2) 사이킷런의 cosine_simlilarity 이용

- 모든 문서 간의 유사도 계산

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
cos_sim_mat = cosine_similarity(ftr_mat, ftr_mat)
cos_sim_mat

array([[1.        , 0.40207758, 0.40425045],
       [0.40207758, 1.        , 0.45647296],
       [0.40425045, 0.45647296, 1.        ]])

### 2. 유클리드 거리를 이용한 유사도

| |바나나|사과|저는|좋아요|
|---|---|---|---|---|
|문서1|2|3|0|1|
|문서2|1|2|3|1|
|문서3|2|1|2|2|
|문서Q|1|1|0|1|

- 코사인 유사도 계산

In [19]:
ftr_mat2 = np.array([[2,3,0,1],
                    [1,2,3,1],
                    [2,1,2,2],
                    [1,1,0,1]])

cos_sim_mat2 = cosine_similarity(ftr_mat2, ftr_mat2)
cos_sim_mat2

array([[1.        , 0.621059  , 0.66712438, 0.9258201 ],
       [0.621059  , 1.        , 0.85933785, 0.59628479],
       [0.66712438, 0.85933785, 1.        , 0.80064077],
       [0.9258201 , 0.59628479, 0.80064077, 1.        ]])

- 유클리드 거리

In [21]:
def euclid_dist(x, y):
    return np.sqrt(np.sum((x - y)**2))

doc1 = ftr_mat2[0]
doc2 = ftr_mat2[1]
doc3 = ftr_mat2[2]
docQ = ftr_mat2[3]

print(f'문서1과 문서2의 유클리드 거리 : {euclid_dist(doc1, doc2):.3f}')
print(f'문서1과 문서3의 유클리드 거리 : {euclid_dist(doc1, doc3):.3f}')
print(f'문서1과 문서Q의 유클리드 거리 : {euclid_dist(doc1, docQ):.3f}')
print(f'문서2과 문서3의 유클리드 거리 : {euclid_dist(doc2, doc3):.3f}')
print(f'문서2과 문서Q의 유클리드 거리 : {euclid_dist(doc2, docQ):.3f}')
print(f'문서3과 문서Q의 유클리드 거리 : {euclid_dist(doc3, docQ):.3f}')

문서1과 문서2의 유클리드 거리 : 3.317
문서1과 문서3의 유클리드 거리 : 3.000
문서1과 문서Q의 유클리드 거리 : 2.236
문서2과 문서3의 유클리드 거리 : 2.000
문서2과 문서Q의 유클리드 거리 : 3.162
문서3과 문서Q의 유클리드 거리 : 2.449


### 3. 자카드 유사도

- 뉴스 기사들의 유사도 탐지
- 추천시스템에서 두 고객간의 유사도

In [22]:
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokens_doc1 = doc1.split()
tokens_doc2 = doc2.split()

print(f'문서1 토큰 : {tokens_doc1}')
print(f'문서2 토큰 : {tokens_doc2}')

문서1 토큰 : ['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
문서2 토큰 : ['apple', 'banana', 'coupon', 'passport', 'love', 'you']


- 문서1과 문서2의 합집합

In [24]:
union = set(tokens_doc1).union(set(tokens_doc2))
print(f'두 문서의 합집합 : {union}')

두 문서의 합집합 : {'everyone', 'like', 'holder', 'passport', 'coupon', 'apple', 'love', 'likey', 'banana', 'you', 'watch', 'card'}


- 문서1과 문서2의 교집합

In [25]:
intersection = set(tokens_doc1).intersection(set(tokens_doc2))
print(f'두 문서의 교집합 : {intersection}')

두 문서의 교집합 : {'banana', 'apple'}


- 문서1과 문서2의 자카드 유사도

In [26]:
print(f'자카드 유사도 : {len(intersection)/len(union):.3f}')

자카드 유사도 : 0.167


- 자카드유사도 함수 정의

In [27]:
def jaccard_similarity(s1, s2):
    s1 = set(s1)
    s2 = set(s2)
    return len(s1 & s1) / len(s1 | s2)

In [28]:
jaccard_similarity(tokens_doc1, tokens_doc1)

1.0

------------